In [1]:
# installing necessary libraries
import selenium
import requests
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import datetime

In [2]:
# ignoring depreciation warnings
import warnings
warnings.filterwarnings("ignore")

### Initialising Browser instance

In [3]:

options = Options()
options.add_argument('start-maximised')
options.add_argument(f"--user-data-dir=C:/Users/amanc/AppData/Local/Google/Chrome/User Data")
options.add_argument(r'--profile-directory=Default')
options.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 0 
})
driver = webdriver.Chrome(options = options, service=Service(ChromeDriverManager().install()))
driver.get('https://pib.gov.in/allRel.aspx')

In [4]:
#Data extraction fucntion
def extract_data(month, df):
    #detecting all articles
    articles = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="form1"]/section/div/div/div/div/ul/li/ul/li/a')))
    for article in articles:
        #obtaining link
        text = requests.get(article.get_attribute('href'))
        data = BeautifulSoup(text.text, 'html.parser')
        paras = data.findAll('p')
        content = ''
        #Extracting written content
        for j in paras:
#             stripping Nonetype, author name etc, whitespaces
            if len(str(j.find(text=True))) > 20:
#         adding each paragraph form the article into the content section
                content += str(j.find(text=True)).strip('\xa0')
        date_text = data.find(class_ = 'ReleaseDateSubHeaddateTime text-center pt20').text
#         splitting for using datetime function
        date = date_text[46:57].split()
#         replacing month name with digit
        date[1] = month

        date = datetime.datetime(int(date[2]), int(date[1]), int(date[0]))
        title = article.get_attribute('title').strip()
        link = article.get_attribute('href')
#         appending entries to DataFrame
        df = df.append({'Date':date, 'Title':title, 'Content':content, 'Link':link}, ignore_index=True)
    return df

### Date Select

In [9]:
# Using DataFrame to optimise time and space complexities
df1 = pd.DataFrame(columns=['Date', 'Title', 'Content', 'Link'])
# parsing 1 through 4 to get first three months
for month in range(1,4):
#     finding and clicking required setting for the article
    try:
        Day = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//select[@id='ContentPlaceHolder1_ddlday']")))
        Day = Day.click()
        all_Days = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//select[@id="ContentPlaceHolder1_ddlday"]/option[1]')))
        all_Days.click()
    except:
        pass
    try:
        Month = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//select[@id='ContentPlaceHolder1_ddlMonth']")))
        Month.click()
        select_month = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//select[@id="ContentPlaceHolder1_ddlMonth"]/option[{month}]')))
        select_month.click()
        time.sleep(3)
    except:
        pass
#     updating datafram
    df1 = extract_data(month, df1)
    


In [12]:
df1

,Date,Title,Content,Link
0,2024-01-31,ADDRESS BY THE HON’BLE PRESIDENT OF India Smt...,1. This is my first address in this new Par...,https://pib.gov.in/PressReleasePage.aspx?PRID=...
1,2024-01-26,PRESIDENT OF INDIA HOSTS PRESIDENT OF FRANCE,"The President of India, Smt Droupadi Murmu rec...",https://pib.gov.in/PressReleasePage.aspx?PRID=...
2,2024-01-25,ADDRESS TO THE NATION BY THE PRESIDENT OF INDI...,"My Dear Fellow Citizens,My heartiest greetings...",https://pib.gov.in/PressReleasePage.aspx?PRID=...
3,2024-01-25,PRESIDENT OF INDIA GRACES THE 14TH NATIONAL VO...,"The President of India, Smt Droupadi Murmu gra...",https://pib.gov.in/PressReleasePage.aspx?PRID=...
4,2024-01-24,PRESIDENT OF THE 78TH SESSION OF THE UNITED NA...,"H.E. Mr. Dennis Francis, the President of the ...",https://pib.gov.in/PressReleasePage.aspx?PRID=...
...,...,...,...,...
3340,2024-03-08,Competition Commission of India notifies three...,The Competition Commission of India (CCI) has ...,https://pib.gov.in/PressReleasePage.aspx?PRID=...
3341,2024-03-06,CCI approves acquisition of 59.25% shareholdin...,The Competition Commission of India (CCI) has ...,https://pib.gov.in/PressReleasePage.aspx?PRID=...
3342,2024-03-05,CCI organises 9th edition of National Conferen...,The Competition Commission of India (CCI) orga...,https://pib.gov.in/PressReleasePage.aspx?PRID=...
3343,2024-03-04,Competition Commission of India to hold 9th Na...,The Competition Commission of India (CCI) will...,https://pib.gov.in/PressReleasePage.aspx?PRID=...


In [19]:
# sorting according to date
df1.sort_values('Date', inplace = True, ignore_index= True)

In [23]:
df1.to_csv('PIB_Jan-Mar.csv')